In [1]:
import os
print(os.getcwd())

import warnings
warnings.filterwarnings(action='ignore')

c:\jupyter_home\통합프로젝트\library\code


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('../data2/library_dataset_loan_description.csv')
display(df.head())
df.info()

,user_key,loan_date,return_date,isbn13,title,author,publisher,add_code,img_url,kdc_class_no,sex,age,description
0,4208654315914400000,2022-03-31 12:00:00.0,2022-04-18 12:00:00.0,9788932105635,주님의 기도:해설,로마노 구아르디니 저;이석은 역,가톨릭출판사,3230.0,http://image.aladin.co.kr/product/29/95/cover/...,233,0.0,0.0,별다른 의미 없이 기도를 반복하는 신자들을 위해 영혼들에게 깨달음을 주는 기도해설서...
1,4208654315914400000,2022-03-31 12:00:00.0,2022-04-18 12:00:00.0,9788932105567,생명의 옹달샘,[가톨릭출판사]편집부 엮음,가톨릭출판사,4230.0,https://bookthumb-phinf.pstatic.net/cover/014/...,234,0.0,0.0,하느님이 주신 주옥 같은 말씀들을 주제별로 엮은 책이다. 하느님의 말씀을 모은 작은...
2,1748935651519910000000000,2022-04-19 12:00:00.0,2022-04-22 12:00:00.0,9788931418828,전자상거래관리사 2급 필기 마스터바이블:전자상거래관련법규,배근성;영진정보연구소 공저,영진닷컴,13000.0,http://image.aladin.co.kr/product/30/30/cover/...,326,0.0,0.0,정보통신부와 산업자원부가 공동으로 주관하고 대한상공회의소에서 시행하는 국가공인 시험...
3,4208654315914400000,2022-03-31 12:00:00.0,2022-04-18 12:00:00.0,9788988800980,이민 가서 뭐해 먹고 살지:뉴질랜드 편,진양경 씀,무한,3810.0,http://image.aladin.co.kr/product/30/25/cover/...,322,0.0,0.0,뉴질랜드에서 할만한 직업을 소개한 책. 뉴질랜드 현지의 직업과 그와 관련된 세부적인...
4,573748883482940000000000,2022-04-19 12:00:00.0,2022-04-28 12:00:00.0,9788931418613,(Bingo!)컴퓨터활용능력 2급 필기:컴퓨터 일반,김옥남;영진정보연구소 공저,영진닷컴,13000.0,http://image.aladin.co.kr/product/30/28/cover/...,4,0.0,0.0,"독학으로 충분히 시험에 대비할 수 있는 폭넓은 이론과 다양한 문제와, 1회부터 현 ..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72088 entries, 0 to 72087
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_key      72088 non-null  object 
 1   loan_date     72088 non-null  object 
 2   return_date   72088 non-null  object 
 3   isbn13        72088 non-null  int64  
 4   title         72088 non-null  object 
 5   author        72088 non-null  object 
 6   publisher     72088 non-null  object 
 7   add_code      72088 non-null  float64
 8   img_url       72088 non-null  object 
 9   kdc_class_no  72088 non-null  int64  
 10  sex           72088 non-null  float64
 11  age           72088 non-null  float64
 12  description   72088 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usage: 7.1+ MB


In [4]:
df = df[['user_key','title','kdc_class_no','description']]

In [21]:
# doc2vec model 학습 및 저장
def doc2vec_models(tagged_data, tok,
                   vector_size=128, window=3, epochs=40, min_count=0, workers=4):
    model = Doc2Vec(tagged_data, vector_size=vector_size, window=window, epochs=epochs, min_count=min_count, workers=workers)
    model.save(f'../models/{tok}_model.doc2vec')

In [7]:
# 데이터 전처리
def preprocessing_data(df):
    df['title_description'] = df['title'] + " " + df['description']
    df.drop(['title', 'description'], axis=1, inplace=True)
    return df

In [8]:
# doc2vec에서 사용할 수 있는 데이터 생성
def doc2vec_data(df, column, t_document=False):
    df_doc = []
    for tag, doc in zip(df.index, df[column]):
        doc = doc.split(" ")
        df_doc.append(([tag], doc))
    if t_document:
        df = [TaggedDocument(words=text, tags=tag) for tag, text in df_doc]
        return df
    else:
        return df_doc

In [43]:
# 추천 결과 반환
def recommened_contents(user, df_doc, model):
    scores = []
    
    for tags, text in df_doc:
        trained_doc_vec = model.docvecs[tags[0]]
        scores.append(cosine_similarity(user.reshape(-1, 128), trained_doc_vec.reshape(-1, 128)))
        
    scores = np.array(scores).reshape(-1)
    scores = np.argsort(-scores)[:2]
    
    return df.loc[scores, :]

In [37]:
# user embedding 생성
def user_embedding(index_list, df_doc, model):
    user=[]
    user_embedding=[]
    for i in index_list:
        user.append(df_doc[i][0][0])
    for i in user:
        user_embedding.append(model.docvecs[i])
    user_embedding = np.array(user_embedding)
    user=np.mean(user_embedding, axis=0)
    return user

In [35]:
# 사용자 히스토리 출력
def view_user_history(df):
    print(df[['kdc_class_no', 'title_description']])

In [12]:
df = preprocessing_data(df)

In [13]:
df.head()

,user_key,kdc_class_no,title_description
0,4208654315914400000,233,주님의 기도:해설 별다른 의미 없이 기도를 반복하는 신자들을 위해 영혼들에게 깨달음...
1,4208654315914400000,234,생명의 옹달샘 하느님이 주신 주옥 같은 말씀들을 주제별로 엮은 책이다. 하느님의 말...
2,1748935651519910000000000,326,전자상거래관리사 2급 필기 마스터바이블:전자상거래관련법규 정보통신부와 산업자원부가 ...
3,4208654315914400000,322,이민 가서 뭐해 먹고 살지:뉴질랜드 편 뉴질랜드에서 할만한 직업을 소개한 책. 뉴질...
4,573748883482940000000000,4,(Bingo!)컴퓨터활용능력 2급 필기:컴퓨터 일반 독학으로 충분히 시험에 대비할 ...


In [3]:
df = pd.read_csv('../data2/description_okt.csv')

In [4]:
df = df[['user_key','kdc_class_no','title_description','okt_nouns']]
df.head()

,user_key,kdc_class_no,title_description,okt_nouns
0,4208654315914400000,233,주님의 기도:해설 별다른 의미 없이 기도를 반복하는 신자들을 위해 영혼들에게 깨달음...,"['주님', '기도', '해설', '의미', '기도', '반복', '신자', '위해..."
1,4208654315914400000,234,생명의 옹달샘 하느님이 주신 주옥 같은 말씀들을 주제별로 엮은 책이다. 하느님의 말...,"['생명', '옹달샘', '하느님', '주신', '주옥', '말씀', '주제', '..."
2,1748935651519910000000000,326,전자상거래관리사 2급 필기 마스터바이블:전자상거래관련법규 정보통신부와 산업자원부가 ...,"['전자상거래', '관리사', '급', '필기', '마스터', '바이블', '전자상..."
3,4208654315914400000,322,이민 가서 뭐해 먹고 살지:뉴질랜드 편 뉴질랜드에서 할만한 직업을 소개한 책. 뉴질...,"['이민', '뭐', '살', '뉴질랜드', '편', '뉴질랜드', '직업', '소..."
4,573748883482940000000000,4,(Bingo!)컴퓨터활용능력 2급 필기:컴퓨터 일반 독학으로 충분히 시험에 대비할 ...,"['컴퓨터', '활용', '능력', '급', '필기', '컴퓨터', '일반', '독..."


In [12]:
df_doc_title_description_tag = doc2vec_data(df, 'title_description', t_document=True)
df_doc_title_description = doc2vec_data(df, 'title_description')
df_doc_tok_tag = doc2vec_data(df, 'okt_nouns', t_document=True)
df_doc_tok = doc2vec_data(df, 'okt_nouns')

### make doc2vec models


In [22]:
doc2vec_models(df_doc_title_description_tag, tok=False)

In [23]:
doc2vec_models(df_doc_tok_tag, tok=True)

### load doc2vec models

In [ ]:
def doc2vec_models(tagged_data, tok,
                   vector_size=128, window=3, epochs=40, min_count=0, workers=4):
    model = Doc2Vec(tagged_data, vector_size=vector_size, window=window, epochs=epochs, min_count=min_count, workers=workers)
    model.save(f'../models/{tok}_model.doc2vec')

In [24]:
model_title_content = Doc2Vec.load('../models/False_model.doc2vec')

In [25]:
model_tok = Doc2Vec.load('../models/True_model.doc2vec')

In [28]:
df = df[['kdc_class_no','title_description','okt_nouns']]

In [30]:
df.head()

,kdc_class_no,title_description,okt_nouns
0,233,주님의 기도:해설 별다른 의미 없이 기도를 반복하는 신자들을 위해 영혼들에게 깨달음...,"['주님', '기도', '해설', '의미', '기도', '반복', '신자', '위해..."
1,234,생명의 옹달샘 하느님이 주신 주옥 같은 말씀들을 주제별로 엮은 책이다. 하느님의 말...,"['생명', '옹달샘', '하느님', '주신', '주옥', '말씀', '주제', '..."
2,326,전자상거래관리사 2급 필기 마스터바이블:전자상거래관련법규 정보통신부와 산업자원부가 ...,"['전자상거래', '관리사', '급', '필기', '마스터', '바이블', '전자상..."
3,322,이민 가서 뭐해 먹고 살지:뉴질랜드 편 뉴질랜드에서 할만한 직업을 소개한 책. 뉴질...,"['이민', '뭐', '살', '뉴질랜드', '편', '뉴질랜드', '직업', '소..."
4,4,(Bingo!)컴퓨터활용능력 2급 필기:컴퓨터 일반 독학으로 충분히 시험에 대비할 ...,"['컴퓨터', '활용', '능력', '급', '필기', '컴퓨터', '일반', '독..."


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72088 entries, 0 to 72087
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   kdc_class_no       72088 non-null  int64 
 1   title_description  72088 non-null  object
 2   okt_nouns          72088 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [47]:
# 사용자 히스토리
user_kdc_322 = df.loc[random.sample(df.loc[df.kdc_class_no == 322, :].index.values.tolist(), 2), :]  
view_user_history(user_kdc_322)

       kdc_class_no                                  title_description
12575           322  신흥시장국의 경기회복 차별화 및 정책과제 본 보고서에서는 중국ㆍ인도ㆍ브라질 등이 빠...
48246           322  한국의 경제 성장과 정부의 역할 :경제적 지대의 추계를 중심으로 우리나라는 짧은 기...


In [48]:
# user embedding
user_1 = user_embedding(user_kdc_322.index.values.tolist(), df_doc_tok, model_tok)

In [49]:
# 책 추천
result = recommened_contents(user_1, df_doc_tok, model_tok)

In [50]:
result

,kdc_class_no,title_description,okt_nouns
12575,322,신흥시장국의 경기회복 차별화 및 정책과제 본 보고서에서는 중국ㆍ인도ㆍ브라질 등이 빠...,"['신흥시장', '국', '경기', '회복', '차별', '및', '정책', '과제..."
48246,322,한국의 경제 성장과 정부의 역할 :경제적 지대의 추계를 중심으로 우리나라는 짧은 기...,"['한국', '경제', '성장', '정부', '역할', '경제', '지대', '추계..."


In [51]:
pd.DataFrame(result.loc[:, ['kdc_class_no', 'title_description']])

,kdc_class_no,title_description
12575,322,신흥시장국의 경기회복 차별화 및 정책과제 본 보고서에서는 중국ㆍ인도ㆍ브라질 등이 빠...
48246,322,한국의 경제 성장과 정부의 역할 :경제적 지대의 추계를 중심으로 우리나라는 짧은 기...
